In [ ]:
import json
import time

from PIL import Image
from io import BytesIO
from base64 import b64decode as decoder
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

In [ ]:
class Spider:
    """
    This is the constructor class to which you can pass a bunch of parameters. 
    These parameters are stored to the class instance variables so that the
    class functions can access them later.
    
    no_of_pages: number of pages to scrape
    """
    def __init__(self, no_of_pages=1):
        self.driver = webdriver.Firefox(
            executable_path=GeckoDriverManager().install()
        )
        self.no_of_pages = no_of_pages
        self.page_list = []
        self.url_list = []
                
    def get_info(self, url, selectors, page_id=1):
        driver = self.driver
        result = dict()
        driver.get(url)
        time.sleep(2)
        
        for each_selector in selectors:
            result[each_selector['alias']] = self.get_target(
                {**each_selector, "page_id":page_id}
            )
        
        return result
    
    def get_target(self, selector):
        driver = self.driver
        assert type(selector) == dict
        page_id = selector['page_id']
        target = selector['target']
        css_selector = selector['css_selector'].format(page_id)
        
        assert (type(css_selector) == str) or (type(css_selector) == list)
        
        if target == 'src':
            try:
                return driver.find_element_by_css_selector(
                    css_selector
                ).get_attribute('src')
            except:
                return None
        else:
            return None

In [ ]:
my_spider = Spider()

In [ ]:
base_url = 'http://appstrillas.mx/pdfFlipping/viewer.jsp?id=SbS3AB#page/{}'

In [ ]:
page_id=50
test_url = base_url.format(page_id)
test_url

In [ ]:
selectors = [{
        'alias': 'img-data',
        'target': 'src',
        'css_selector': '.p{} > img:nth-child(2)',
}]

In [ ]:
my_product = my_spider.get_info(test_url, selectors, page_id)

In [ ]:
img_raw = my_product['img-data'].replace("data:image/jpg;base64,","")

In [ ]:
img = Image.open(BytesIO(decoder(img_raw)))

In [ ]:
img

In [ ]:
img.save('img/page-{}.jpg'.format(page_id),'JPEG')